In [17]:
from ultralytics import YOLO
import numpy as np
from PIL import Image

img2D = np.asarray(Image.open('../floor2.jpg'))
model = YOLO('yolov8x-seg.pt')
decreaseByProbability = True

In [19]:
import numpy as np
from PIL import Image

def detectObjects(img2D, model, decreaseByProbability):
    results = model.predict(source=img2D, conf=0.01)
    costmap = np.ones((img2D.shape[0], img2D.shape[1]))
    
    numberOfBoxes = results[0].boxes.shape[0]
    boxes = results[0].boxes.numpy()
    probabilities = boxes.conf
    
    for box in range(numberOfBoxes):
        currBox = boxes.xyxy[box]
        x1 = currBox[0].astype(int)
        x2 = currBox[2].astype(int)
        y1 = currBox[1].astype(int)
        y2 = currBox[3].astype(int)

        if decreaseByProbability:
            decreaseAmount = probabilities[box]
        else:
            decreaseAmount = 1

        for x in range(x1, x2 + 1):
            for y in range(y1, y2 + 1):
                costmap[y][x] -= decreaseAmount
                
    return costmap

In [20]:
costmapResult = detectObjects(img2D, model, decreaseByProbability)


0: 640x480 1 knife, 2 cell phones, 9003.2ms
Speed: 846.9ms preprocess, 9003.2ms inference, 343.7ms postprocess per image at shape (1, 3, 640, 480)


In [21]:
image = Image.fromarray(costmapResult * 255).convert("L")
image.save("costmap.png")